In [25]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import matplotlib.pyplot as plt
import pandas as pd
from shapely import geos, wkt
from shapely.wkt import loads

In [3]:
df = pd.read_excel('accidents.xlsx', sheet_name = 'ONSV')
#df.head()

In [4]:
vehicle_columns = ['VEHICULO 1', 'VEHICULO 2', 'VEHICULO 3', 'VEHICULO 4']

def count_non_empty(row):
    return sum(1 for col in vehicle_columns if pd.notna(row[col]))

In [5]:
df['# of vehicles in accident'] = df.apply(count_non_empty, axis=1)
df.head()

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,GRAVEDAD,TOTAL FALLECIDOS,TOTAL LESIONADOS,VEHICULO 1,VEHICULO 2,VEHICULO 3,VEHICULO 4,LATITUD,LONGITUD,# of vehicles in accident
0,1,UPIAT - HUACHO,LIMA,HUAURA,VEGUETA,10/01/2021,2021,January,Sunday,05:20,...,FATAL,1,0,NO IDENTIFICADO,NaN,NaN,NaN,-11.030084,-77.592648,1
1,2,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,10/01/2021,2021,January,Sunday,20:30,...,FATAL,1,1,AUTOMOVIL,AUTOMOVIL,NaN,NaN,-11.605631,-77.241772,2
2,3,UPIAT - HUACHO,LIMA,HUAURA,HUACHO,12/01/2021,2021,January,Tuesday,19:39,...,FATAL,1,0,AUTOMOVIL,AUTOMOVIL,AUTOMOVIL,NaN,-11.184285,-77.576240,3
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,FATAL,1,0,REMOLCADOR - SEMIREMOLQUE,NaN,NaN,NaN,-12.054372,-77.142617,1
4,5,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,24/01/2021,2021,January,Sunday,20:10,...,FATAL,1,0,OMNIBUS,BICICLETA,NaN,NaN,-11.881892,-77.126265,2


In [6]:
# Convert the DataFrame to a GeoDataFrame with LineString geometry
gdf_rta = gpd.GeoDataFrame(df)

In [7]:
gdf_rta['geometry'] = gpd.points_from_xy(gdf_rta['LONGITUD'], gdf_rta['LATITUD'])

In [8]:
gdf_rta = gdf_rta.drop(columns=['LONGITUD', 'LATITUD'])

In [9]:
gdf_rta.head()

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,TIPO DE SINIESTRO,GRAVEDAD,TOTAL FALLECIDOS,TOTAL LESIONADOS,VEHICULO 1,VEHICULO 2,VEHICULO 3,VEHICULO 4,# of vehicles in accident,geometry
0,1,UPIAT - HUACHO,LIMA,HUAURA,VEGUETA,10/01/2021,2021,January,Sunday,05:20,...,ATROPELLO FUGA,FATAL,1,0,NO IDENTIFICADO,NaN,NaN,NaN,1,POINT (-77.59265 -11.03008)
1,2,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,10/01/2021,2021,January,Sunday,20:30,...,CHOQUE,FATAL,1,1,AUTOMOVIL,AUTOMOVIL,NaN,NaN,2,POINT (-77.24177 -11.60563)
2,3,UPIAT - HUACHO,LIMA,HUAURA,HUACHO,12/01/2021,2021,January,Tuesday,19:39,...,CHOQUE,FATAL,1,0,AUTOMOVIL,AUTOMOVIL,AUTOMOVIL,NaN,3,POINT (-77.57624 -11.18429)
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,CAÍDA DE PASAJERO,FATAL,1,0,REMOLCADOR - SEMIREMOLQUE,NaN,NaN,NaN,1,POINT (-77.14262 -12.05437)
4,5,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,24/01/2021,2021,January,Sunday,20:10,...,DESPISTE,FATAL,1,0,OMNIBUS,BICICLETA,NaN,NaN,2,POINT (-77.12627 -11.88189)


In [10]:
gdf_rta.crs = 'EPSG:4326'

In [45]:
df2 = pd.read_excel('results_viasegura.xlsx')
df2.head()

,section_id,analisis_name,geometry,distances,bicycle_facility,bicycle_facility_confidence,carriageway,carriageway_confidence,delineation,delineation_confidence,...,skid_resistance,skid_resistance_confidence,speed_management,speed_management_confidence,street_lighting,street_lighting_confidence,upgrade_cost,upgrade_cost_confidence,vehicle_parking,vehicle_parking_confidence
0,0,PE20230416M-F-P05,"(-77.13728466666667 -12.048542333333334, -77.1...",100.042404,Not Present,0.878739,Undivided road,0.878739,Poor,0.878739,...,Sealed - adequate,0.878739,Not present,0.878739,Present,0.878739,High,0.878739,Unknown,0.878739
1,1,PE20230416M-F-P05,"(-77.13813941814638 -12.048873240439098, -77.1...",99.797630,Not Present,0.949207,Undivided road,0.949207,Poor,0.949207,...,Sealed - medium,0.949207,Present,0.949207,Not present,0.949207,High,0.949207,Unknown,0.949207
2,2,PE20230416M-F-P05,"(-77.13893224806256 -12.049322386887004, -77.1...",100.303366,Not Present,0.701126,Undivided road,0.701126,Poor,0.701126,...,Sealed - adequate,0.701126,Present,0.701126,Present,0.701126,Low,0.701126,Unknown,0.701126
3,3,PE20230416M-F-P05,"(-77.13961848742838 -12.049919145919157, -77.1...",99.798441,Not Present,0.853882,Undivided road,0.853882,Adequate,0.853882,...,Sealed - adequate,0.853882,Present,0.853882,Not present,0.853882,High,0.853882,Unknown,0.853882
4,4,PE20230416M-F-P05,"(-77.14018322611551 -12.050621615007024, -77.1...",99.910489,Not Present,0.778106,Undivided road,0.778106,Poor,0.778106,...,Sealed - adequate,0.778106,Present,0.778106,Not present,0.778106,High,0.778106,Unknown,0.778106


In [50]:
def create_linestring(coords):
    point_coords = [tuple(map(float, coord.split())) for coord in coords.strip('()').split(',')]
    points = [Point(x, y) for x, y in point_coords]
    return LineString(points)

In [51]:
df2['linestring_column'] = df2['geometry'].apply(create_linestring)

ValueError: not enough values to unpack (expected 2, got 1)

In [44]:
# Convert the DataFrame to a GeoDataFrame with LineString geometry
gdf_defects = gpd.GeoDataFrame(df2, geometry = df2['geometry'])

TypeError: Input must be valid geometry objects: LINESTRING (-77.13728466666667 -12.048542333333334, -77.13728747791089 -12.048543316713229, -77.13729028915512 -12.048544300093123, -77.13729310039933 -12.048545283473018, -77.13729591164355 -12.048546266852913, -77.13729872288776 -12.048547250232808, -77.13730153413199 -12.048548233612703, -77.13730434537621 -12.048549216992598, -77.13730715662042 -12.048550200372492, -77.13730996786465 -12.048551183752387, -77.13731277910887 -12.048552167132282, -77.13731559035308 -12.048553150512177, -77.1373184015973 -12.048554133892072, -77.13732121284153 -12.048555117271967, -77.13732402408574 -12.048556100651862, -77.13732683532996 -12.048557084031756, -77.13732964657419 -12.048558067411651, -77.1373324578184 -12.048559050791546, -77.13733526906262 -12.048560034171441, -77.13733808030683 -12.048561017551336, -77.13734089155106 -12.04856200093123, -77.13734370279528 -12.048562984311125, -77.13734651403949 -12.04856396769102, -77.13734932528372 -12.

In [29]:
df2['geometry'] = gpd.GeoSeries.to_wkt(df2['geometry'])


ValueError: 'geoms' must be a GeometryArray

In [80]:
gdf_defects.columns

Index(['analisis_name', 'id_clase', 'clases', 'id_tramo_falla', 'distances',
       'coordenada_inicio', 'latitud_inicio', 'longitud_inicio',
       'coordenada_final', 'latitude_final', 'longitud_final', 'base', 'area',
       'boxes', 'geometry'],
      dtype='object')

In [81]:
gdf_defects = gdf_defects.drop(columns=['coordenada_inicio','coordenada_final', 'latitude_final', 'longitud_final'])

In [82]:
gdf_defects.head()

,analisis_name,id_clase,clases,id_tramo_falla,distances,latitud_inicio,longitud_inicio,base,area,boxes,geometry
0,PE20230403T-F-P06,0,Desenfoque Linea Blanca,36,14.636535,-11.044373,-77.589136,910.592651,13327.920895,1,POINT (-77.58914 -11.04437)
1,PE20230403T-F-P06,0,Desenfoque Linea Blanca,37,15.642927,-11.063438,-77.588205,792.107620,12390.881812,1,POINT (-77.58821 -11.06344)
2,PE20230403T-F-P06,0,Desenfoque Linea Blanca,38,0.433942,-11.071721,-77.584876,590.992355,256.456536,1,POINT (-77.58488 -11.07172)
3,PE20230403T-F-P06,0,Desenfoque Linea Blanca,39,0.430801,-11.073484,-77.584176,909.705734,391.902126,1,POINT (-77.58418 -11.07348)
4,PE20230403T-F-P06,0,Desenfoque Linea Blanca,40,0.407377,-11.087819,-77.583247,765.128403,311.695672,1,POINT (-77.58325 -11.08782)


In [83]:
gdf_defects.crs = 'EPSG:4326'

In [84]:
gdf_rta = gdf_rta.to_crs('EPSG:3857')
gdf_defects = gdf_defects.to_crs('EPSG:3857')


In [85]:
print(gdf_rta.crs)
print(gdf_defects.crs)

EPSG:3857
EPSG:3857


In [86]:
gdf_rta['geometry'] = gdf_rta['geometry'].buffer(10)
gdf_rta.head()

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,TIPO DE SINIESTRO,GRAVEDAD,TOTAL FALLECIDOS,TOTAL LESIONADOS,VEHICULO 1,VEHICULO 2,VEHICULO 3,VEHICULO 4,# of vehicles in accident,geometry
0,1,UPIAT - HUACHO,LIMA,HUAURA,VEGUETA,10/01/2021,2021,January,Sunday,05:20,...,ATROPELLO FUGA,FATAL,1,0,NO IDENTIFICADO,NaN,NaN,NaN,1,"POLYGON ((-8637564.096 -1235518.571, -8637564...."
1,2,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,10/01/2021,2021,January,Sunday,20:30,...,CHOQUE,FATAL,1,1,AUTOMOVIL,AUTOMOVIL,NaN,NaN,2,"POLYGON ((-8598504.764 -1300859.092, -8598504...."
2,3,UPIAT - HUACHO,LIMA,HUAURA,HUACHO,12/01/2021,2021,January,Tuesday,19:39,...,CHOQUE,FATAL,1,0,AUTOMOVIL,AUTOMOVIL,AUTOMOVIL,NaN,3,"POLYGON ((-8635737.574 -1253011.901, -8635737...."
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,CAÍDA DE PASAJERO,FATAL,1,0,REMOLCADOR - SEMIREMOLQUE,NaN,NaN,NaN,1,"POLYGON ((-8587466.815 -1351896.932, -8587466...."
4,5,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,24/01/2021,2021,January,Sunday,20:10,...,DESPISTE,FATAL,1,0,OMNIBUS,BICICLETA,NaN,NaN,2,"POLYGON ((-8585646.577 -1332269.933, -8585646...."


In [95]:
#blackspot_analysis = gpd.sjoin(gdf_defects, gdf_rta, predicate='within', lsuffix= '_defects', rsuffix = '_fatalities')
blackspot_analysis = gpd.sjoin(gdf_rta, gdf_defects, predicate='contains')

In [99]:
blackspot_analysis.head(100)

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,analisis_name,id_clase,clases,id_tramo_falla,distances,latitud_inicio,longitud_inicio,base,area,boxes
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,PE20230416M-F-P06,0,Desenfoque Linea Blanca,13,0.173064,-12.054305,-77.142675,387.975769,67.144734,1
9,10,UPIAT - HUACHO,LIMA,HUAURA,SANTA MARIA,04/03/2021,2021,March,Thursday,06:28,...,PE20230403M-F-P11,0,Desenfoque Linea Blanca,5,17.207931,-11.112783,-77.592410,672.329636,11569.402301,1
19,20,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,25/04/2021,2021,April,Sunday,02:15,...,PE20230416M-F-P05,0,Piel de Cocodrilo,3,45.278309,-12.050022,-77.139718,907.700394,46203.012815,8
21,22,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,29/04/2021,2021,April,Thursday,16:00,...,PE20230416M-F-P03,0,Desenfoque Linea Blanca,103,11.170553,-11.994462,-77.126014,247.783813,2767.882215,1
25,26,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,17/05/2021,2021,May,Monday,16:10,...,PE20230416M-F-P02,0,"Protuberancia, Bache",99,12.214683,-11.904137,-77.128520,207.583008,2535.560716,1
27,28,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,05/06/2021,2021,June,Saturday,19:30,...,PE20230416M-F-P03,0,Intervalo Lineal Transversal,80,0.340532,-11.964844,-77.126666,148.770447,50.661024,1
34,35,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,25/06/2021,2021,June,Friday,05:35,...,PE20230416M-F-P09,0,Piel de Cocodrilo,25,19.019896,-11.873081,-77.127180,915.230236,16241.884542,3
53,54,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,07/11/2021,2021,November,Sunday,04:30,...,PE20230403M-F-P04,0,Piel de Cocodrilo,39,13.466999,-11.595672,-77.254922,867.635765,12036.385361,2
59,60,UPIAT - HUACHO,LIMA,BARRANCA,SUPE,27/11/2021,2021,November,Saturday,01:10,...,PE20230403M-F-P14,0,Desenfoque Linea Blanca,37,15.957515,-10.802256,-77.703451,467.552376,7460.974056,1
75,76,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,22/01/2022,2022,January,Saturday,06:00,...,PE20230416M-F-P06,0,Grieta Lineal Transversal,125,7.483147,-12.046644,-77.132144,203.663177,1524.041538,1


In [100]:
blackspot_analysis.to_csv("blackspot_analysis.csv")